<a href="https://colab.research.google.com/github/MMatsuokaM/Embeddings_IATechniques_imersaoAula05./blob/main/embeddings_ia_techniques_imersaoAula05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U google-generativeai

In [4]:
import numpy as np
import pandas as pd

import google.generativeai as genai
from google.colab import userdata
api_key = userdata.get('SECRET_KEY')
genai.configure(api_key=api_key)

In [21]:
#Listagem de documentos que serão buscados
DOCUMENT1 = {
    "Titulo": "Few-Shot Prompting",
    "Conteudo": "É uma técnica que permite a modelos de linguagem aprenderem novas tarefas com poucos exemplos, imitando o aprendizado humano a partir de informações limitadas."}

DOCUMENT2 = {
    "Titulo": "Chain-of-Thought Prompting",
    "Conteudo": "É uma técnica para melhorar o raciocínio de modelos de linguagem, pedindo que eles expliquem seu processo de pensamento passo-a-passo, como um humano faria."}

DOCUMENT3 = {
    "Titulo": "Self-Consistency",
    "Conteudo": "É uma técnica que aumenta a confiabilidade de modelos de linguagem, gerando múltiplas soluções para um problema e selecionando a resposta mais consistente entre elas."}

DOCUMENT4 = {
    "Titulo": "Zero-Shot Prompting",
    "Conteudo": "É a capacidade de um modelo de linguagem realizar tarefas sem exemplos específicos de treinamento, apenas com instruções gerais."
}

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3, DOCUMENT4]

In [22]:
df = pd.DataFrame(documents)
df.columns = ["Titulo", "Conteudo"]
df

,Titulo,Conteudo
0,Few-Shot Prompting,É uma técnica que permite a modelos de linguag...
1,Chain-of-Thought Prompting,É uma técnica para melhorar o raciocínio de mo...
2,Self-Consistency,É uma técnica que aumenta a confiabilidade de ...
3,Zero-Shot Prompting,É a capacidade de um modelo de linguagem reali...


In [23]:
model = "models/embedding-001"

In [24]:
def embed_fn(title, text):
  return genai.embed_content(model=model,
                                 content=text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")["embedding"]
df

,Titulo,Conteudo
0,Few-Shot Prompting,É uma técnica que permite a modelos de linguag...
1,Chain-of-Thought Prompting,É uma técnica para melhorar o raciocínio de mo...
2,Self-Consistency,É uma técnica que aumenta a confiabilidade de ...
3,Zero-Shot Prompting,É a capacidade de um modelo de linguagem reali...


In [25]:
df["Embeddings"] = df.apply(lambda row: embed_fn(row["Titulo"], row["Conteudo"]), axis=1)
df

,Titulo,Conteudo,Embeddings
0,Few-Shot Prompting,É uma técnica que permite a modelos de linguag...,"[0.014087488, -0.023800643, -0.037058447, 0.03..."
1,Chain-of-Thought Prompting,É uma técnica para melhorar o raciocínio de mo...,"[0.007122226, -0.06846033, -0.05094824, 0.0324..."
2,Self-Consistency,É uma técnica que aumenta a confiabilidade de ...,"[0.04307626, -0.07189356, -0.03094445, 0.03441..."
3,Zero-Shot Prompting,É a capacidade de um modelo de linguagem reali...,"[-0.004718392, -0.05182432, -0.05416365, 0.009..."


In [26]:
def gerar_e_buscar_consulta(consulta, base, model):
  embedding_da_consulta = genai.embed_content(model=model,
                                 content=consulta,
                                 task_type="RETRIEVAL_QUERY")["embedding"]

  produtos_escalares = np.dot(np.stack(df["Embeddings"]), embedding_da_consulta)

  indice = np.argmax(produtos_escalares)
  return df.iloc[indice]["Titulo"]

In [27]:
consulta = "Qual técnica de IA aprenderem novas tarefas?"

trecho = gerar_e_buscar_consulta(consulta, df, model)
print(trecho)

Few-Shot Prompting


In [28]:
consulta = "Qual técnica de IA é mais confiável?"

trecho = gerar_e_buscar_consulta(consulta, df, model)
print(trecho)

Self-Consistency


In [29]:
consulta = "Qual técnica de IA obtenho resposta mais coerentes?"

trecho = gerar_e_buscar_consulta(consulta, df, model)
print(trecho)

Self-Consistency


In [30]:
consulta = "Qual técnica de IA é usada para chatbot?"

trecho = gerar_e_buscar_consulta(consulta, df, model)
print(trecho)

Chain-of-Thought Prompting


In [36]:
consulta = "Qual técnica de IA é precisa só de informações existentes?"

trecho = gerar_e_buscar_consulta(consulta, df, model)
print(trecho)


Zero-Shot Prompting


In [19]:
generation_config = {
  "temperature": 0,
  "candidate_count": 1
}

In [37]:
prompt = f"Reescreva esse texto de uma forma mais descontraída, sem adicionar informações que não façam parte do texto: {trecho}"

model_2 = genai.GenerativeModel("gemini-1.0-pro",
                                generation_config=generation_config)
response = model_2.generate_content(prompt)
print(response.text)

**Zero-Shot Prompting: O jeito ninja de fazer IA entender**

Imagine que você tem uma IA superinteligente, mas ela só sabe fazer o que você manda. E se você pudesse dar comandos mais criativos, sem precisar ensinar tudo do zero?

É aí que entra o Zero-Shot Prompting. É como dar à sua IA um manual de instruções secreto, dizendo: "Ei, aqui estão alguns exemplos do que eu quero que você faça. Use sua imaginação para entender o que eu quero dizer."

Assim, a IA pode entender comandos como "Escreva uma história sobre um gato que salva o mundo" ou "Traduza este texto para Klingon" sem nunca ter visto esses comandos antes.

É como se você estivesse ensinando um ninja a lutar. Você não precisa mostrar cada movimento, basta dar alguns exemplos e o ninja entende o resto.

Então, da próxima vez que você quiser que sua IA faça algo novo, não se preocupe em programá-la. Basta usar o Zero-Shot Prompting e deixe a IA fazer o trabalho pesado.
